In [ ]:
from dask import delayed
from dask.distributed import Client

client = Client(n_workers=4)

In [ ]:
from preprocess import Preprocessor

import ml_inference
from ml_inference import (
    BaselineRegressor, AutoRegressor, cross_val_plot, error_plot
)

import pandas as pd
import seaborn as sns

pd.options.mode.chained_assignment = None
INFILE = '../data/pennycook_et_al_study2_clean.csv'

In [ ]:
df = pd.read_csv(INFILE)
X, treat, y = df.drop(columns='Diff'), df.Treatment, df.Diff
df.head()

In [ ]:
from sklearn.linear_model import LinearRegression


class BaselineTreatmentRegressor(LinearRegression):
    def __init__(self, treatment_var):
        self.treatment_var = treatment_var
        super().__init__()
        
    def fit(self, X, y, sample_weight=None):
        return super().fit(self.transform_X(X), y, sample_weight)
        
    def predict(self, X):
        return super().predict(self.transform_X(X))
    
    def transform_X(self, X):
        return (
            X[self.treatment_var].values if isinstance(X, pd.DataFrame)
            else X[:, self.treatment_var]
        ).reshape(-1, 1)

In [ ]:
def get_treatment_idx(treatment_var, X):
    if not isinstance(X, pd.DataFrame):
         X = pd.DataFrame(X)
    return pd.get_dummies(X[treatment_var])

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score

class TreatmentRegressor():
    def __init__(self, models, treatment_var, scoring=r2_score):
        self.models = models
        self.treatment_arms = list(models.keys())
        self.treatment_var = treatment_var
        self.scoring = scoring
        
    def fit(self, X, y):
        def fit_weights(X, y):
            X_t = self.transform_X(X)
            self.constrained_weight_ = fit_constrained_weight(X_t, y)
            resid = y - (X_t * self.constrained_weight_).sum(axis=1)
            split = treatment_split(self.treatment_var, X, [X_t, resid])
            self.weight_ = pd.DataFrame(
                {key: fit_unconstrained_weight(X, resid) 
                for key, (_, X, resid) in split.items()}
            )[self.treatment_arms]
            
        def fit_constrained_weight(X, y):
            # constrain weights to sum to 1
            reference = X.columns[0]
            y -= X[reference]
            X = X.apply(lambda x: x-X[reference]).drop(columns=reference)
            reg = LinearRegression(fit_intercept=False).fit(X, y)
            return pd.Series(
                index=self.treatment_arms,
                data=np.insert(reg.coef_, 0, 1-reg.coef_.sum())
            )
        
        def fit_unconstrained_weight(X, y):
            # residual weights sum to 0
            # INCREASE THE WEIGHT UNTIL ALL COEFS ARE BETWEEN 0 AND 1
            # make this the lower bound
            reg = Ridge(500, fit_intercept=False).fit(X, y)
            return (reg.coef_ - reg.coef_.mean()) + self.constrained_weight_

        X, y = X.reset_index(drop=True), y.reset_index(drop=True)
        # store means and stds for each treatment arm
        self.mean_ = y.groupby(X[self.treatment_var]).mean()
        self.std_ = y.groupby(X[self.treatment_var]).std()
        # standardize y for each condition
        treatment_idx = self.get_treatment_idx(X)
        y = (y - treatment_idx @ self.mean_) / (treatment_idx @ self.std_)
        # train models for each treatment arm
        split = treatment_split(self.treatment_var, X, y)
        [self.models[key].fit(X, y) for key, (X, y) in split.items()]
        # fit weights
        fit_weights(X, y)
        return self
    
    def get_treatment_idx(self, X):
        idx = get_treatment_idx(self.treatment_var, X)
        return idx[self.treatment_arms].reset_index(drop=True)

    def predict(self, X, constrained=False):
        treatment_idx = self.get_treatment_idx(X)
        X = self.transform_X(X)
        if constrained:
            y_pred = (self.constrained_weight_ * X).sum(axis=1)
        else:
            y_pred = ((treatment_idx @ self.weight_.T) * X).sum(axis=1)
        return y_pred * (treatment_idx @ self.std_) + treatment_idx @ self.mean_
    
    def predict_effect(self, X, control_val=0, treatment_val=1):
        def unnormalize(df, value):
            df[value] = df[value] * self.std_[value] + self.mean_[value]
            
        X = self.transform_X(X)
        y_pred = X @ self.weight_[[control_val, treatment_val]]
        unnormalize(y_pred, control_val)
        unnormalize(y_pred, treatment_val)
        return y_pred[treatment_val] - y_pred[control_val]
        
    def transform_X(self, X):
        X = (
            X.drop(columns=self.treatment_var) if isinstance(X, pd.DataFrame) 
            else np.delete(X, self.treatment_var, axis=1)
        )
        return pd.DataFrame({
            key: model.predict(X) for key, model in self.models.items()
        }).reset_index()[self.treatment_arms]
    
    def score(self, X, y, constrained=False, *args, **kwargs):
        return self.scoring(y, self.predict(X, constrained), *args, **kwargs)
    
    def get_params(self, deep=True):
        params = dict(
            models=self.models,
            treatment_var=self.treatment_var,
            scoring=self.scoring
        )
#         if deep:
#             control_params = self.control_model.get_params(deep)
#             treat_params = self.treat_model.get_params(deep)
#             params.update({'control__'+key: val for key, val in control_params.items()})
#             params.update({'treat__'+key: val for key, val in treat_params.items()})
        return params
    
    def set_params(self, **params):
        self.models = params['models']
        self.treatment_var = params['treatment_var']
        self.scorer = params['scorer']

def explain_effect(reg, X, nsamples=1000, local=True, scoring=r2_score, scoring_params={}):
    if local:
        g = lambda effect_pred: effect_pred
    else:
        effect = reg.predict_effect(X)
        g = lambda effect_pred: scoring(effect, effect_pred, **scoring_params)
    explainer = gshap.KernelExplainer(reg.predict_effect, X, g)
    gshap_values = explainer.gshap_values(X, nsamples=nsamples)
    if local:
        if isinstance(X, pd.DataFrame):
            return pd.DataFrame(columns=X.columns, data=gshap_values.T)
        return gshap_values.T
    gshap_values /= gshap_values.sum()
    if isinstance(X, pd.DataFrame):
        return pd.DataFrame({'Feature': X.columns, 'G-SHAP': gshap_values})
    return gshap_values

In [ ]:
def treatment_split(treatment_var, X, vectors=[], rm_treatment=True):
    def split_vectors(idx):
        if not vectors:
            return split_vector(X, idx)
        return [split_vector(v, idx) for v in [X]+vectors]
    
    def split_vector(v, idx):
        return (
            v[idx==1].reset_index(drop=True) if isinstance(v, (pd.DataFrame, pd.Series))
            else v[idx]
        )
    
    if not isinstance(vectors, list):
        vectors = [vectors]
    treatment_idx = get_treatment_idx(treatment_var, X)
    if rm_treatment:
        if isinstance(X, pd.DataFrame):
            # treat_var is a column name
            X = X.drop(columns=treatment_var)
        else:
            # treat_var is a column index
            X = np.delete(X, treatment_var, axis=1)
    return {key: split_vectors(idx) for key, idx in treatment_idx.iteritems()}

In [ ]:
def train_autoreg(treatment, X, y):
    print('\nTuning model for treatment arm', treatment)
    return AutoRegressor(preprocess=Preprocessor(X), n_jobs=-1).tune(X, y, n_iter=2**5)

split = treatment_split('Treatment', X, y)
models = {key: train_autoreg(key, X, y) for key, (X, y) in split.items()}

In [ ]:
reg = TreatmentRegressor(models, 'Treatment')
reg.fit(X, y)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

def make_svr_pipeline():
    return make_pipeline(
        Preprocessor(X),
        SVR()
    )

models = {
    0: make_svr_pipeline(),
    1: make_svr_pipeline()
}
reg = TreatmentRegressor(models, 'Treatment')
reg.fit(X, y)

In [ ]:
effect = reg.predict_effect(X)
sns.histplot(effect, kde=True)
effect.describe()

In [ ]:
weight_df = reg.weight_.copy()
weight_df = weight_df.apply(lambda x: x-reg.constrained_weight_)
ax = sns.heatmap(weight_df, center=0, annot=True)
ax.set(xlabel='Treatment group', ylabel='Weight on treatment-specific model')

In [ ]:
%%time

from sklearn.model_selection import KFold, cross_val_score

scores = []
for i in range(10):
    score = {'C': [], 'UC': []}
    kf = KFold(10, shuffle=True)
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        reg.fit(X_train, y_train)
        score['UC'].append(reg.score(X_test, y_test, constrained=False))
        score['C'].append(reg.score(X_test, y_test, constrained=True))
    score = {key: np.array(val).mean() for key, val in score.items()}
    scores.append(score)

In [ ]:
tmp_df = pd.DataFrame(scores)
tmp_df['Diff'] = tmp_df['UC'] - tmp_df['C']
tmp_df

In [ ]:
ttest_1samp(tmp_df['Diff'], 0)

In [ ]:
from scipy.stats import ttest_1samp

from copy import deepcopy

In [ ]:
%%time

def moderation_test(reg, X, y, repeat=10, cv=10):
    def compute_scores(kf, reg, X, y):
        score = {'Unconstrained': [], 'Constrained': []}
        # compute score for all folds
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            reg = delayed(reg.fit)(X_train, y_train)
            score['Unconstrained'].append(
                delayed(reg.score)(X_test, y_test, constrained=False)
            )
            score['Constrained'].append(
                delayed(reg.score)(X_test, y_test, constrained=True)
            )
        # return mean CV score
        return delayed(dict)(
            Unconstrained=np.array(score['Unconstrained']).mean(), 
            Constrained=np.array(score['Constrained']).mean()
        ).compute()
        
    scores = []
    for _ in range(repeat):
        kf = KFold(cv, shuffle=True)
        scores.append(compute_scores(kf, reg, X, y))
    scores = pd.DataFrame(scores)
    scores['Delta'] = scores['Unconstrained'] - scores['Constrained']
    res = ttest_1samp(df['Diff'], 0)
    # convert p-value for 1-tailed hypothesis
    pvalue = res.pvalue/2 if res.statistic > 0 else (1-res.pvalue/2)
    return scores, res.statistic, pvalue

scores, t, p = moderation_test(reg, X, y)

In [ ]:
%%time

from sklearn.base import clone

def moderation_test(reg, X, y, repeat=10, cv=10):
    def compute_scores(kf, reg, X, y):
#         reg = deepcopy(reg)
        reg = clone(reg)
        X, y = X.copy(), y.copy()
        score = {'Unconstrained': [], 'Constrained': []}
        # compute score for all folds
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            reg = delayed(reg.fit)(X_train, y_train)
            score['Unconstrained'].append(
                delayed(reg.score)(X_test, y_test, constrained=False)
            )
            score['Constrained'].append(
                delayed(reg.score)(X_test, y_test, constrained=True)
            )
        # return mean CV score
        return dict(
            Unconstrained=np.array(score['Unconstrained']).mean(), 
            Constrained=np.array(score['Constrained']).mean()
        )
        
    scores = []
    for _ in range(repeat):
        kf = KFold(cv, shuffle=True)
        scores.append(compute_scores(kf, reg, X, y))
    scores = delayed(pd.DataFrame)(scores).compute()
    scores['Diff'] = scores['Unconstrained'] - scores['Constrained']
    res = ttest_1samp(scores['Diff'], 0)
    # convert p-value for 1-tailed hypothesis
    pvalue = res.pvalue/2 if res.statistic > 0 else (1-res.pvalue/2)
    return scores, res.statistic, pvalue

scores, t, p = moderation_test(reg, X, y, 10, 10)

In [ ]:
scores

In [ ]:
t, p